# RMI Detail Transactions Project

## 1.  Load Transactional Data, SalesForce Data, Mavenlink Data, load packages, and convert to Dataframes

I am using two reports from Netsuite to populate this data:  Lists/Employees --> Run report without filters all set to "All".

Search in Netsuite search bar usings "sea" for Adaptive Transaction Search - P&L - Jeremy

![title](img/picture.jpg)

In [1]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles.borders import Border, Side


In [2]:
from xml.sax import ContentHandler, parse

# Reference https://goo.gl/KaOBG3
class ExcelHandler(ContentHandler):
    def __init__(self):
        self.chars = [  ]
        self.cells = [  ]
        self.rows = [  ]
        self.tables = [  ]
    def characters(self, content):
        self.chars.append(content)
    def startElement(self, name, atts):
        if name=="Cell":
            self.chars = [  ]
        elif name=="Row":
            self.cells=[  ]
        elif name=="Table":
            self.rows = [  ]
    def endElement(self, name):
        if name=="Cell":
            self.cells.append(''.join(self.chars))
        elif name=="Row":
            self.rows.append(self.cells)
        elif name=="Table":
            self.tables.append(self.rows)

excelHandler = ExcelHandler()
parse('../Data/raw/ADAPTIVETransactionSearchPLJeremyResults849.xls', excelHandler)
df1 = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])



In [3]:
excelHandler = ExcelHandler()
parse('../Data/raw/MavenlinkTimeAuditResults82.xls', excelHandler)
dfMavenlink = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

In [4]:
excelHandler = ExcelHandler()
parse('../Data/raw/Employees621.xls', excelHandler)
employee_list = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])


In [5]:
dfSalesforce = pd.read_html('../Data/raw/report1614354395009.xls')
dfSalesforce = dfSalesforce[0]

## 2. Data Manipulation

In this section I start manipulting these datasets into somethign more digestable for the end user.  The main manipulation I am doing in this seciton is pulling information out of the "Memo" field that users may want to see seperately.

In [6]:
EmployeeNameandID = employee_list[['Name', 'External ID', 'Job Title']]

In [7]:
EmployeeNameandID['External ID'].replace('', np.nan, inplace=True)

C:\Users\Jeremy Wendt\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [8]:
len(EmployeeNameandID)

497

In [9]:
#EmployeeNameandID.to_excel('originallist.xlsx')

In [10]:
employee_id_list = employee_list['External ID'].tolist()

In [11]:
employee_id_list = list(filter(None, employee_id_list))

In [12]:
print(len(employee_id_list))

453


In [13]:
df1['Memo'] = df1['Memo'].astype('string')

In [14]:
df1.dtypes

Date                               object
Period                             object
GL Account                         object
Document Number                    object
Program                            object
Initiative                         object
Project                            object
Restriction                        object
Grant                              object
Amount                             object
Vendor Name                        object
Employee Name                      object
Memo                               string
Amount (Credit)                    object
Amount (Debit)                     object
Client                             object
Transaction Number                 object
Type                               object
Vendor Rate                        object
Vendor Contract Expiration Date    object
dtype: object

In [15]:
all_ids = '|'.join(employee_id_list)

In [16]:
def return_first(x):
    try:
        y = x[0]
    except:
        y = ""
    return y

In [17]:
df1['External ID'] = df1['Memo'].str.findall(all_ids).apply(return_first)

In [18]:
employee_name_list = employee_list['Name'].tolist()

In [19]:
employee_name_list = list(filter(None, employee_name_list))

In [20]:
all_names = '|'.join(employee_name_list)

In [21]:
all_names = all_names + "|Agnis Li|Anna Ledden|Greg Hales|Rachel Hohensee|Li Ma"

In [22]:
df1['Match ID Names'] = df1['Memo'].str.findall(all_names).apply(return_first)

In [23]:
df1 = pd.merge(df1, EmployeeNameandID, on = 'External ID', how='left')

In [24]:
df1['Match ID Names'].replace('', np.nan, inplace=True)

In [25]:
df1['Name'].replace('', np.nan, inplace=True)

In [26]:
df1['Name'].fillna(df1['Match ID Names'], inplace=True)

In [27]:
df1['Name'].replace('Agnis Li', 'Agnes Li', inplace=True)
df1['Name'].replace('Anna Ledden', 'Anna  Ledden', inplace=True)
df1['Name'].replace('Rachel Hohensee', 'Rachel T Hohensee', inplace=True)

In [28]:
len(df1)

84467

## Mavenlink Data Manipulation

In [29]:
dfMavenlink['Month'] = pd.DatetimeIndex(dfMavenlink['Date']).month
dfMavenlink['Month'] = dfMavenlink['Month'].map({7: 'Jul 2020', 8: "Aug 2020", 9: "Sep 2020", 10: "Oct 2020", 11: "Nov 2020", 12: "Dec 2020", 1: "Jan 2021", 2: "Feb 2021", 3: "Mar 2021", 4: "Apr 2021", 5: "May 2021", 6: "Jun 2021"})

In [30]:
dfMavenlink['Minutes'] =  dfMavenlink['Duration'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]) * 1)
dfMavenlink['Hours'] =  dfMavenlink['Minutes'] / 60

In [31]:
dfMavenlink['Grant'].replace('', np.nan, inplace=True)
dfMavenlink['Initiative'].replace('', np.nan, inplace=True)
dfMavenlink['Employee'].replace('', np.nan, inplace=True)
dfMavenlink['Case/Task/Event'].replace('', np.nan, inplace=True)
dfMavenlink['Billable'].replace('', np.nan, inplace=True)

In [32]:
dfMavenlink['Grant'].fillna("No Grant", inplace=True)
dfMavenlink['Initiative'].fillna("No Initiative", inplace=True)
dfMavenlink['Grant'].fillna("Employee", inplace=True)
dfMavenlink['Case/Task/Event'].fillna("Case/Task/Event", inplace=True)
dfMavenlink['Billable'].fillna("Billable", inplace=True)

In [33]:
#dfMavenlink.to_excel('Output_Mavenlink.xlsx')

## Salesforce Data Manipulation

In [34]:
dfSalesforce['Revenue Status'].replace('', np.nan, inplace=True)
dfSalesforce['Initiative'].replace('', np.nan, inplace=True)
dfSalesforce['Revenue Amount'].replace('', np.nan, inplace=True)

In [35]:
dfSalesforce['Revenue Status'].fillna("No Status", inplace=True)
dfSalesforce['Initiative'].fillna("No Initiative", inplace=True)
dfSalesforce['Revenue Amount'].fillna(0, inplace=True)

In [36]:
dfSalesforce.to_excel('Output_Salesforce.xlsx')

## Extract Time Allocation From Memo

In [37]:
allocations = ['BONUS ALLOCATION', 'ADMIN ALLOCATION', 'PTO/HOLIDAY ALLOCATION']

In [38]:
all_allocations = '|'.join(allocations)

In [39]:
df1['ALLOCATION'] = df1['Memo'].str.findall(all_allocations).apply(return_first)

## Extract Project Fields for Heather's report

In [40]:
df1['Project Allocations'] = df1['Project'].str.findall(r'\[(.*?)\]').apply(return_first)

## Add Rollup Accounts

In [41]:
lookups = pd.read_excel('../Data/raw/lookups.xlsx')

In [42]:
lookups.head()

,GL Account,Rollup Account
0,5010 Contributions Revenue : Foundation Contri...,5000 Contributions Revenue
1,5015 Contributions Revenue : Campaign Foundati...,5000 Contributions Revenue
2,5020 Contributions Revenue : Individual Contri...,5000 Contributions Revenue
3,5025 Contributions Revenue : Campaign Corporat...,5000 Contributions Revenue
4,5030 Contributions Revenue : Endowment Contrib...,5000 Contributions Revenue


In [43]:
df1 = pd.merge(df1, lookups, on = 'GL Account', how='left')

In [44]:
df1['Account Digit'] = df1['GL Account'].astype('string').str[0]

In [45]:
df1.columns

Index(['Date', 'Period', 'GL Account', 'Document Number', 'Program',
       'Initiative', 'Project', 'Restriction', 'Grant', 'Amount',
       'Vendor Name', 'Employee Name', 'Memo', 'Amount (Credit)',
       'Amount (Debit)', 'Client', 'Transaction Number', 'Type', 'Vendor Rate',
       'Vendor Contract Expiration Date', 'External ID', 'Match ID Names',
       'Name', 'Job Title', 'ALLOCATION', 'Project Allocations',
       'Rollup Account', 'Account Digit'],
      dtype='object')

In [46]:
df1['Amount'] = df1['Amount'].astype(float)
df1['Amount (Credit)'].replace('', 0, inplace=True)
df1['Amount (Debit)'].replace('', 0, inplace=True)
df1['Amount (Credit)'] = df1['Amount (Credit)'].astype(float)
df1['Amount (Debit)'] = df1['Amount (Debit)'].astype(float)

In [47]:
df1.dtypes

Date                                object
Period                              object
GL Account                          object
Document Number                     object
Program                             object
Initiative                          object
Project                             object
Restriction                         object
Grant                               object
Amount                             float64
Vendor Name                         object
Employee Name                       object
Memo                                string
Amount (Credit)                    float64
Amount (Debit)                     float64
Client                              object
Transaction Number                  object
Type                                object
Vendor Rate                         object
Vendor Contract Expiration Date     object
External ID                         object
Match ID Names                      object
Name                                object
Job Title  

## Add Reclass Field and Closeout field

In [48]:
account_exceptions = ['To reclass', 'Reclass']

In [49]:
Accounting_Moves = '|'.join(account_exceptions)

In [50]:
closeout = 'close out'

In [51]:
df1['Accounting Adjustments'] = df1['Memo'].str.findall(Accounting_Moves).apply(return_first)

In [52]:
df1['Accounting Close Out'] = df1['Memo'].str.findall(closeout).apply(return_first)

In [53]:
df1['Vendor Contract Expiration Date'] = pd.to_datetime(df1['Vendor Contract Expiration Date'])

In [54]:
df1['Vendor Contract Expiration Date'] = df1['Vendor Contract Expiration Date'].dt.date

## ADDING TITLES INSTEAD OF NAMES

In [55]:
EmployeeNameandID['Job Title'].value_counts()

Senior Associate                      73
Associate                             63
Manager                               63
Intern                                47
Principal                             38
                                      ..
Maintenance Technician                 1
HR Manager                             1
Director of Learning & Development     1
Machine Learning Scientist             1
Multilateral Coordinator               1
Name: Job Title, Length: 111, dtype: int64

In [56]:
uniquetitles = EmployeeNameandID['Job Title'].unique()

In [57]:
Titleslist = uniquetitles.tolist()

In [58]:
def createEmployeeTitlesdf(title, df):
    newdf = df[df['Job Title']==title]
    newdf.reset_index(drop=True, inplace=True)
    newdf['PositionID'] = newdf['Job Title'] + newdf.index.astype(str)    
    return newdf.to_dict()

In [59]:
DictList = {}

for i in range(len(Titleslist)):
    newdict = createEmployeeTitlesdf(Titleslist[i], EmployeeNameandID)
    DictList[i] = newdict

<ipython-input-58-933c5a1db0ab>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['PositionID'] = newdf['Job Title'] + newdf.index.astype(str)


In [60]:
dflist = []

for i in range(len(DictList)):
    df = pd.DataFrame(DictList[i])
    dflist.append(df)

In [61]:
finalemployeedf = pd.concat(dflist).reset_index(drop=True)

In [62]:
employeelisttomerge = finalemployeedf[['Name', 'PositionID']]

In [63]:
#employeelisttomerge.to_excel('listwithposition.xlsx')

In [64]:
len(employeelisttomerge)

497

In [65]:
df1 = pd.merge(df1, employeelisttomerge, on = 'Name', how='left')

In [66]:
df1['Initiative'].replace('', np.nan, inplace=True)
df1['Project'].replace('', np.nan, inplace=True)
df1['Grant'].replace('', np.nan, inplace=True)
df1['GL Account'].replace('', np.nan, inplace=True)
df1['Memo'].replace('', np.nan, inplace=True)
df1['Transaction Number'].replace('', np.nan, inplace=True)
df1['Vendor Name'].replace('', np.nan, inplace=True)
df1['Vendor Rate'].replace('', np.nan, inplace=True)
df1['Vendor Contract Expiration Date'].replace('', np.nan, inplace=True)
df1['Program'].replace('', np.nan, inplace=True)
df1['Project Allocations'].replace('', np.nan, inplace=True)
df1['Project Allocations'].replace('e', np.nan, inplace=True)
df1['Accounting Adjustments'].replace('To reclass', 'Reclass', inplace=True)
df1['Client'].replace('', np.nan, inplace=True)

In [67]:
df1['Initiative'].fillna("No Initiative", inplace=True)
df1['Project'].fillna("No Project", inplace=True)
df1['Grant'].fillna("No Grant", inplace=True)
df1['GL Account'].fillna("No GL Account", inplace=True)
df1['Memo'].fillna("No Memo", inplace=True)
df1['Transaction Number'].fillna("No Transaction Number", inplace=True)
df1['Vendor Name'].fillna("No Vendor Name", inplace=True)
df1['Vendor Rate'].fillna("No Vendor Rate", inplace=True)
df1['Vendor Contract Expiration Date'].fillna("No Contract Expiration Date", inplace=True)
df1['Program'].fillna("No Program", inplace=True)
df1['Project Allocations'].fillna("Other", inplace=True)
df1['Client'].fillna("No Client", inplace=True)

## Add Further Heather Calcs

In [68]:
df1['Inst or Reclass'] = df1['Account Digit'] + df1['Accounting Adjustments']

In [69]:
df1['Inst or Reclass'].value_counts()

7           65736
8           11527
5            4446
6            1852
7Reclass      694
8Reclass      165
5Reclass       46
Name: Inst or Reclass, dtype: Int64

In [70]:
df1['Inst or Reclass'] = df1['Inst or Reclass'].map({'7': "Expense", '8': "Institutional Overhead", '5': "Revenue", '6': "Revenue", '7Reclass': "Expense Reclass", '8Reclass': "Inst Overhead Reclass", '5Reclass': "Revenue Reclass", '6Reclass': "Revenue Reclass"})

## Unrestricted Columns Added

In [71]:
def return_grant(x):
    if x != "No Grant":
        return "Yes"
    else: 
        return "No"
    
    

In [72]:
df1['grant?'] = df1['Grant'].apply(return_grant)

In [73]:
df1["Restricted Revenue"] = df1['grant?'] + df1['Account Digit']

In [74]:
df1["Restricted Revenue"] = df1["Restricted Revenue"].map({'No7': "No", 'Yes7': "Restricted Revenue", 'No8': "No", 'No5': "No", 'Yes8': "Restricted Revenue", 'Yes6': "No", 'No6': "No", 'Yes5': "No"})

In [75]:
df1['Restricted Revenue'] = np.where(df1['Restricted Revenue'] == "Restricted Revenue", df1['Amount'], 0)

In [76]:
df1["Rev or Exp"] = df1["Account Digit"].map({'6': 'Revenue', '5': 'Revenue', '7': 'Expense', '8': 'Expense'})

In [77]:
df1["Rev or Exp"].value_counts()

Expense    78122
Revenue     6344
Name: Rev or Exp, dtype: int64

In [78]:
df1['Multilateral'] = np.where(df1['Rev or Exp'] == "Revenue", df1['Amount'], 0)

In [79]:
df1['General Expense'] = np.where(df1['Rev or Exp'] == "Expense", df1['Amount'], 0)

In [80]:
df1['Unrestricted'] = - df1['Restricted Revenue'] - df1['Multilateral'] + df1['General Expense']

In [81]:
df1 = df1.drop('Match ID Names',1)

## Output manipulated DF to Excel

In [82]:
#df1.to_excel('Output_Netsuite.xlsx')

# 3.  Data Segmentation by Program

Here I'm taking the manipulated data and ultimately creating a dataframe to filter out each program's restricted revenue.

In [83]:
Programs = list(df1['Program'].unique())

In [84]:
print(Programs)

['Development', 'Islands', 'India', 'Operations', 'China', 'Breakthrough Technology', 'Carbon-Free Mobility', 'Climate Aligned Industries', 'Global Climate Finance', 'Carbon-Free Electricity', 'Strategic Engagement and Analysis Group', 'Carbon-Free Buildings', 'WattTime', 'Urban Transformation', 'Climate Intelligence', 'Industry', 'Africa', 'SE Asia', 'US', 'Building Electrification', 'Communications', 'Energy Transition Academy', 'No Program']


In [85]:
def create_program(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[firstdf['Grant'] != 'No Grant'] 
    thirddf = seconddf[(seconddf['Account Digit']=='7')|(seconddf['Account Digit']=='8')]
    if thirddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return thirddf

In [86]:
Development = create_program('Development')
Islands = create_program('Islands')
India = create_program('India')
Operations = create_program('Operations')
China = create_program('China')
Breakthrough_Technology = create_program('Breakthrough Technology')
CarbonFree_Mobility = create_program('Carbon-Free Mobility')
Climate_Aligned_Industries = create_program('Climate Aligned Industries')
Global_Climate_Finance = create_program('Global Climate Finance')
Strategic_Engagement_and_Analysis_Group = create_program('Strategic Engagement and Analysis Group')
CarbonFree_Buildings = create_program('Carbon-Free Buildings')
WattTime = create_program('WattTime')
CarbonFree_Electricty = create_program('Carbon-Free Electricity')
Urban_Transformation = create_program('Urban Transformation')
Climate_Intelligence = create_program('Climate Intelligence')
Industry = create_program('Industry')
SE_Asia = create_program('SE Asia')
Africa = create_program('Africa')
US = create_program('US')
Building_Electrification = create_program('Building Electrification')



Development DF is EMPTY


In [87]:
Developing_Economies_Rollup = pd.concat([Islands, Africa, SE_Asia], ignore_index = True, sort=False)
CarbonFreeBuildings_Rollup = pd.concat([CarbonFree_Buildings, Building_Electrification], ignore_index = True, sort=False)

## 4. Create Reports Path and Manipulate File Names

I will need to come up with naming conventions that work over time for this project, so in this step I do just that.   Also I am loading the Openpyxl packages that i'm going to need for the rest of this program.

In [88]:
import os

path = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\reports\Detail_Trans_Reports'

In [89]:
from datetime import datetime
today = datetime.today()
datem = datetime(today.year, today.month-1, 1)
timestamp = datem.strftime("%b_%Y")

In [90]:
print(timestamp+".xlsx")

Feb_2021.xlsx


In [91]:
from openpyxl.styles import Color, PatternFill, Font, Border
greyFill = PatternFill(start_color='00C0C0C0',
                   end_color='00C0C0C0',
                   fill_type='solid')

## 5.  Create Pivot Table, Add Subtotals, Export to Excel, and Combine All These Functions into One

In [92]:
def create_multi_pivot(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant', 'GL Account', 'Memo', 'Transaction Number', 'Vendor Name', 'Vendor Rate', 'Vendor Contract Expiration Date'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [93]:
def add_subtotal_multi(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', '', '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '', '', '', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [94]:
def to_excel(pivot, str):
    pivot.to_excel(path+str, freeze_panes = (1,9), engine = 'openpyxl', sheet_name = 'Restricted_Rev_Detail', float_format = "%.2f")

In [95]:


def edit_workbook(file):
    book = load_workbook(file)
    sheet = book.active #active means last opened sheet
    sheet['A1']= "Initiative"
    sheet['B1']= "Project"
    sheet['C1']= "Grant"
    sheet['D1']= "GL Account"
    sheet['E1']= "Memo"
    sheet['F1']= "Transaction Number"
    sheet['G1']= "Vendor Name"
    sheet['H1']= "Vendor Rate"
    sheet['I1']= "Vendor Contract Expiration Date" 
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet['G1'].font = Font(bold=True)
    sheet['H1'].font = Font(bold=True)
    sheet['I1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5   
    sheet.column_dimensions['U'].width = 12.5   
    sheet.column_dimensions['V'].width = 12.5   
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(10, 23):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 10):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=9).value == "Subtotal":
            for j in range(10,23):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
#    sheet.protection.sheet = True
    book.save(file)

## Testing for unmerging cells

In [96]:
# THIS CODE WILL UNMERGE ALL THE CELLS IN A SHEET.   
def unmerge_rows(file):
    for col in range(1,5):
        rowstounmerge = []
        book = load_workbook(file)
        testsheet = book['Restricted_Rev_Detail'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                rowstounmerge.append(i)
        rowstounmerge.append(testsheet.max_row)
        for i in range(1,len(rowstounmerge)-1):        
            try:
                testsheet.unmerge_cells(start_row=rowstounmerge[i], start_column=col, end_row = rowstounmerge[i+1]-1, end_column=col) 
            except:
                pass
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['Restricted_Rev_Detail'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                continue
            else:
                testsheet.cell(row=i, column=col).value = testsheet.cell(row=i-1, column=col).value
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['Restricted_Rev_Detail'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=9).value == "Subtotal":
                testsheet.cell(row=i, column=col).value = None
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['Restricted_Rev_Detail'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=1).value == "Grand":
                testsheet.cell(row=i, column=2).value = None
                testsheet.cell(row=i, column=3).value = None
                testsheet.cell(row=i, column=4).value = None
        book.save(file) 

In [97]:
def format_multi_pivot(Group, str):
    step1 = create_multi_pivot(Group)
    step2 = add_subtotal_multi(step1)
    to_excel(step2, str)
    edit_workbook(path+str)
    unmerge_rows(path+str)

In [98]:
#unmerge_rows(path+'\\Strategic_Engagement_and_Analysis_Group_'+timestamp+".xlsx")
#unmerge_rows(path+'\\CarbonFree_Buildings_'+timestamp+".xlsx")

## 6.  Create Restricted Revenue Files

In [99]:
%%time 
'''format_multi_pivot(India, '\\India_'+timestamp+".xlsx")
format_multi_pivot(Operations, '\\Operations_'+timestamp+".xlsx")
format_multi_pivot(China, '\\China_'+timestamp+".xlsx")
format_multi_pivot(Breakthrough_Technology, '\\Breakthrough_Technology-'+timestamp+".xlsx")
format_multi_pivot(CarbonFree_Mobility, '\\CarbonFree_Mobility_'+timestamp+".xlsx")
format_multi_pivot(Climate_Aligned_Industries, '\\Climate_Aligned_Industries_'+timestamp+".xlsx")
format_multi_pivot(Global_Climate_Finance, '\\Global_Climate_Finance_'+timestamp+".xlsx")
format_multi_pivot(Strategic_Engagement_and_Analysis_Group, '\\Strategic_Engagement_and_Analysis_Group_'+timestamp+".xlsx")
format_multi_pivot(CarbonFreeBuildings_Rollup, '\\CarbonFree_Buildings_'+timestamp+".xlsx")
format_multi_pivot(WattTime, '\\WattTime_'+timestamp+".xlsx")
format_multi_pivot(CarbonFree_Electricty, '\\CarbonFree_Electricity_'+timestamp+".xlsx")
format_multi_pivot(Urban_Transformation, '\\Urban_Transformation_'+timestamp+".xlsx")
format_multi_pivot(Climate_Intelligence, '\\Climate_Intelligence_'+timestamp+".xlsx")
format_multi_pivot(Industry, '\\Industry_'+timestamp+".xlsx")
format_multi_pivot(Developing_Economies_Rollup, '\\Developing_Economies_'+timestamp+".xlsx")
format_multi_pivot(US, '\\US_'+timestamp+".xlsx")  '''


Wall time: 0 ns


'format_multi_pivot(India, \'\\India_\'+timestamp+".xlsx")\nformat_multi_pivot(Operations, \'\\Operations_\'+timestamp+".xlsx")\nformat_multi_pivot(China, \'\\China_\'+timestamp+".xlsx")\nformat_multi_pivot(Breakthrough_Technology, \'\\Breakthrough_Technology-\'+timestamp+".xlsx")\nformat_multi_pivot(CarbonFree_Mobility, \'\\CarbonFree_Mobility_\'+timestamp+".xlsx")\nformat_multi_pivot(Climate_Aligned_Industries, \'\\Climate_Aligned_Industries_\'+timestamp+".xlsx")\nformat_multi_pivot(Global_Climate_Finance, \'\\Global_Climate_Finance_\'+timestamp+".xlsx")\nformat_multi_pivot(Strategic_Engagement_and_Analysis_Group, \'\\Strategic_Engagement_and_Analysis_Group_\'+timestamp+".xlsx")\nformat_multi_pivot(CarbonFreeBuildings_Rollup, \'\\CarbonFree_Buildings_\'+timestamp+".xlsx")\nformat_multi_pivot(WattTime, \'\\WattTime_\'+timestamp+".xlsx")\nformat_multi_pivot(CarbonFree_Electricty, \'\\CarbonFree_Electricity_\'+timestamp+".xlsx")\nformat_multi_pivot(Urban_Transformation, \'\\Urban_Transf

## 7.  Add ML and Consulting Fee Tab

In [100]:
def create_df_FeeforService(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[(firstdf['Account Digit']=='5')|(firstdf['Account Digit']=='6')]
    if seconddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return seconddf

In [101]:
DevelopmentFFS = create_df_FeeforService("Development")
IslandsFFS = create_df_FeeforService("Islands")
IndiaFFS = create_df_FeeforService("India")
OperationsFFS = create_df_FeeforService("Operations")
ChinaFFS = create_df_FeeforService("China")
Breakthrough_Technology_FFS = create_df_FeeforService("Breakthrough Technology")
CarbonFree_MobilityFFS = create_df_FeeforService("Carbon-Free Mobility")
ClimateAligned_IndustriesFFS = create_df_FeeforService("Climate Aligned Industries")
CommunicationsFFS = create_df_FeeforService("Communications")
GlobalClimate_FinanceFFS = create_df_FeeforService("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupFFS = create_df_FeeforService("Strategic Engagement and Analysis Group")
Carbon_Free_BuildingsFFS = create_df_FeeforService("Carbon-Free Buildings")
Watttime_FFS = create_df_FeeforService("WattTime")
Carbon_Free_ElectricityFFS = create_df_FeeforService("Carbon-Free Electricity")
UrbanTrans_FFS = create_df_FeeforService("Urban Transformation")
ClimateIntelFFS = create_df_FeeforService("Climate Intelligence")
IndustryFFS = create_df_FeeforService("Industry")
SEAsiaFFS = create_df_FeeforService("SE Asia")
AfricaFFS = create_df_FeeforService("Africa")
USFFS = create_df_FeeforService("US")
BuildingElectrificationFFS = create_df_FeeforService("Building Electrification")

Communications DF is EMPTY


In [102]:
Developing_Economies_RollupFFS = pd.concat([IslandsFFS, AfricaFFS, SEAsiaFFS], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupFFS = pd.concat([Carbon_Free_BuildingsFFS, BuildingElectrificationFFS], ignore_index = True, sort=False)

In [103]:
def create_FFS_pivot(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant', 'GL Account', 'Memo', 'Client', 'Transaction Number'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [104]:
def add_subtotal_FFS(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', '',  'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '',  '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [105]:
def to_excel_FFS(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="ML & FFS Revenue")
    writer.save()
    writer.close()    

In [106]:
def edit_workbook_FFS(file):
    book = load_workbook(file)
    sheet = book["ML & FFS Revenue"] #active means last opened sheet
    sheet['A1']= "Initiative"
    sheet['B1']= "Project"
    sheet['C1']= "Grant"
    sheet['D1']= "GL Account"
    sheet['E1']= "Memo"
    sheet['F1']= "Client"
    sheet['G1']= "Transaction Number" 
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet['G1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5   
    sheet.column_dimensions['U'].width = 12.5   
    sheet.column_dimensions['V'].width = 12.5   
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(8, 21):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 8):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=7).value == "Subtotal":
            for j in range(8,21):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
    sheet.freeze_panes = "H2"
    book.save(file)

In [107]:
def unmerge_rows_FFS(file):
    for col in range(1,5):
        rowstounmerge = []
        book = load_workbook(file)
        testsheet = book['ML & FFS Revenue'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                rowstounmerge.append(i)
        rowstounmerge.append(testsheet.max_row)
        for i in range(1,len(rowstounmerge)-1):        
            try:
                testsheet.unmerge_cells(start_row=rowstounmerge[i], start_column=col, end_row = rowstounmerge[i+1]-1, end_column=col) 
            except:
                pass
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['ML & FFS Revenue'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                continue
            else:
                testsheet.cell(row=i, column=col).value = testsheet.cell(row=i-1, column=col).value
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['ML & FFS Revenue'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=7).value == "Subtotal":
                testsheet.cell(row=i, column=col).value = None
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['ML & FFS Revenue'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=1).value == "Grand":
                testsheet.cell(row=i, column=2).value = None
                testsheet.cell(row=i, column=3).value = None
                testsheet.cell(row=i, column=4).value = None
        book.save(file) 

In [108]:
def format_FFS_pivot(Group, str):
    step1 = create_FFS_pivot(Group)
    step2 = add_subtotal_FFS(step1)
    to_excel_FFS(step2, path+str)
    edit_workbook_FFS(path+str)
    unmerge_rows_FFS(path+str)

## 8.  Add ML and FFS Pivots to Second Tab

In [109]:
'''format_FFS_pivot(Developing_Economies_RollupFFS, '\\Developing_Economies_'+timestamp+".xlsx")
format_FFS_pivot(IndiaFFS, '\\India_'+timestamp+".xlsx")
format_FFS_pivot(OperationsFFS, '\\Operations_'+timestamp+".xlsx")
format_FFS_pivot(ChinaFFS, '\\China_'+timestamp+".xlsx")
format_FFS_pivot(Breakthrough_Technology_FFS, '\\Breakthrough_Technology-'+timestamp+".xlsx")
format_FFS_pivot(CarbonFree_MobilityFFS, '\\CarbonFree_Mobility_'+timestamp+".xlsx")
format_FFS_pivot(ClimateAligned_IndustriesFFS, '\\Climate_Aligned_Industries_'+timestamp+".xlsx")
format_FFS_pivot(GlobalClimate_FinanceFFS, '\\Global_Climate_Finance_'+timestamp+".xlsx")
format_FFS_pivot(Strategic_Engagement_and_Analysis_GroupFFS, '\\Strategic_Engagement_and_Analysis_Group_'+timestamp+".xlsx")
format_FFS_pivot(CarbonFreeBuildings_RollupFFS, '\\CarbonFree_Buildings_'+timestamp+".xlsx")
format_FFS_pivot(Watttime_FFS, '\\WattTime_'+timestamp+".xlsx")
format_FFS_pivot(Carbon_Free_ElectricityFFS, '\\CarbonFree_Electricity_'+timestamp+".xlsx")
format_FFS_pivot(UrbanTrans_FFS, '\\Urban_Transformation_'+timestamp+".xlsx")
format_FFS_pivot(ClimateIntelFFS, '\\Climate_Intelligence_'+timestamp+".xlsx")
format_FFS_pivot(IndustryFFS, '\\Industry_'+timestamp+".xlsx")
format_FFS_pivot(USFFS, '\\US_'+timestamp+".xlsx")'''

'format_FFS_pivot(Developing_Economies_RollupFFS, \'\\Developing_Economies_\'+timestamp+".xlsx")\nformat_FFS_pivot(IndiaFFS, \'\\India_\'+timestamp+".xlsx")\nformat_FFS_pivot(OperationsFFS, \'\\Operations_\'+timestamp+".xlsx")\nformat_FFS_pivot(ChinaFFS, \'\\China_\'+timestamp+".xlsx")\nformat_FFS_pivot(Breakthrough_Technology_FFS, \'\\Breakthrough_Technology-\'+timestamp+".xlsx")\nformat_FFS_pivot(CarbonFree_MobilityFFS, \'\\CarbonFree_Mobility_\'+timestamp+".xlsx")\nformat_FFS_pivot(ClimateAligned_IndustriesFFS, \'\\Climate_Aligned_Industries_\'+timestamp+".xlsx")\nformat_FFS_pivot(GlobalClimate_FinanceFFS, \'\\Global_Climate_Finance_\'+timestamp+".xlsx")\nformat_FFS_pivot(Strategic_Engagement_and_Analysis_GroupFFS, \'\\Strategic_Engagement_and_Analysis_Group_\'+timestamp+".xlsx")\nformat_FFS_pivot(CarbonFreeBuildings_RollupFFS, \'\\CarbonFree_Buildings_\'+timestamp+".xlsx")\nformat_FFS_pivot(Watttime_FFS, \'\\WattTime_\'+timestamp+".xlsx")\nformat_FFS_pivot(Carbon_Free_ElectricityFF

# 9.  Total Expenses

In [110]:
def create_df_allExpenses(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[(firstdf['Account Digit']=='7')|(firstdf['Account Digit']=='8')]
    if seconddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return seconddf

In [111]:
DevelopmentExp = create_df_allExpenses("Development")
IslandsExp = create_df_allExpenses("Islands")
IndiaExp = create_df_allExpenses("India")
OperationsExp = create_df_allExpenses("Operations")
ChinaExp = create_df_allExpenses("China")
Breakthrough_Technology_Exp = create_df_allExpenses("Breakthrough Technology")
CarbonFree_MobilityExp = create_df_allExpenses("Carbon-Free Mobility")
ClimateAligned_IndustriesExp = create_df_allExpenses("Climate Aligned Industries")
CommunicationsExp = create_df_allExpenses("Communications")
GlobalClimate_FinanceExp = create_df_allExpenses("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupExp = create_df_allExpenses("Strategic Engagement and Analysis Group")
Carbon_Free_BuildingsExp = create_df_allExpenses("Carbon-Free Buildings")
Watttime_Exp = create_df_allExpenses("WattTime")
Carbon_Free_ElectricityExp = create_df_allExpenses("Carbon-Free Electricity")
UrbanTrans_Exp = create_df_allExpenses("Urban Transformation")
ClimateIntelExp = create_df_allExpenses("Climate Intelligence")
IndustryExp = create_df_allExpenses("Industry")
SEAsiaExp = create_df_allExpenses("SE Asia")
AfricaExp = create_df_allExpenses("Africa")
USExp = create_df_allExpenses("US")
BuildingElectrificationExp = create_df_allExpenses("Building Electrification")

In [112]:
Developing_Economies_RollupExp = pd.concat([IslandsExp, AfricaExp, SEAsiaExp], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupExp = pd.concat([Carbon_Free_BuildingsExp, BuildingElectrificationExp], ignore_index = True, sort=False)

In [113]:
def create_pivot_Exp(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant', 'GL Account', 'Memo', 'Transaction Number', 'Vendor Name', 'Vendor Rate', 'Vendor Contract Expiration Date'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [114]:
def add_subtotal_Exp(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', '', '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '', '', '', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [115]:
def to_excel_Exp(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="All Expense Detail")
    writer.save()
    writer.close()    

In [116]:
def edit_workbook_Exp(file):
    book = load_workbook(file)
    sheet = book['All Expense Detail'] #active means last opened sheet
    sheet['A1']= "Initiative"
    sheet['B1']= "Project"
    sheet['C1']= "Grant"
    sheet['D1']= "GL Account"
    sheet['E1']= "Memo"
    sheet['F1']= "Transaction Number"
    sheet['G1']= "Vendor Name"
    sheet['H1']= "Vendor Rate"
    sheet['I1']= "Vendor Contract Expiration Date" 
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet['G1'].font = Font(bold=True)
    sheet['H1'].font = Font(bold=True)
    sheet['I1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5   
    sheet.column_dimensions['U'].width = 12.5   
    sheet.column_dimensions['V'].width = 12.5   
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(10, 23):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 10):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=9).value == "Subtotal":
            for j in range(10,23):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
    sheet.freeze_panes = "J2"
    book.save(file)

In [117]:
def unmerge_rows_EXP(file):
    for col in range(1,5):
        rowstounmerge = []
        book = load_workbook(file)
        testsheet = book['All Expense Detail'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                rowstounmerge.append(i)
        rowstounmerge.append(testsheet.max_row)
        for i in range(1,len(rowstounmerge)-1):        
            try:
                testsheet.unmerge_cells(start_row=rowstounmerge[i], start_column=col, end_row = rowstounmerge[i+1]-1, end_column=col) 
            except:
                pass
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['All Expense Detail'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                continue
            else:
                testsheet.cell(row=i, column=col).value = testsheet.cell(row=i-1, column=col).value
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['All Expense Detail'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=9).value == "Subtotal":
                testsheet.cell(row=i, column=col).value = None
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['All Expense Detail'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=1).value == "Grand":
                testsheet.cell(row=i, column=2).value = None
                testsheet.cell(row=i, column=3).value = None
                testsheet.cell(row=i, column=4).value = None
        book.save(file) 

In [118]:
def format_Exp_pivot(Group, str):
    step1 = create_pivot_Exp(Group)
    step2 = add_subtotal_Exp(step1)
    to_excel_Exp(step2, path+str)
    edit_workbook_Exp(path+str)
    unmerge_rows_EXP(path+str)

In [119]:
'''format_Exp_pivot(Developing_Economies_RollupExp, '\\Developing_Economies_'+timestamp+".xlsx")
format_Exp_pivot(IndiaExp, '\\India_'+timestamp+".xlsx")
format_Exp_pivot(OperationsExp, '\\Operations_'+timestamp+".xlsx")
format_Exp_pivot(ChinaExp, '\\China_'+timestamp+".xlsx")
format_Exp_pivot(Breakthrough_Technology_Exp, '\\Breakthrough_Technology-'+timestamp+".xlsx")
format_Exp_pivot(CarbonFree_MobilityExp, '\\CarbonFree_Mobility_'+timestamp+".xlsx")
format_Exp_pivot(ClimateAligned_IndustriesExp, '\\Climate_Aligned_Industries_'+timestamp+".xlsx")
format_Exp_pivot(GlobalClimate_FinanceExp, '\\Global_Climate_Finance_'+timestamp+".xlsx")
format_Exp_pivot(Strategic_Engagement_and_Analysis_GroupExp, '\\Strategic_Engagement_and_Analysis_Group_'+timestamp+".xlsx")
format_Exp_pivot(CarbonFreeBuildings_RollupExp, '\\CarbonFree_Buildings_'+timestamp+".xlsx")
format_Exp_pivot(Watttime_Exp, '\\WattTime_'+timestamp+".xlsx")
format_Exp_pivot(Carbon_Free_ElectricityExp, '\\CarbonFree_Electricity_'+timestamp+".xlsx")
format_Exp_pivot(UrbanTrans_Exp, '\\Urban_Transformation_'+timestamp+".xlsx")
format_Exp_pivot(ClimateIntelExp, '\\Climate_Intelligence_'+timestamp+".xlsx")
format_Exp_pivot(IndustryExp, '\\Industry_'+timestamp+".xlsx")
format_Exp_pivot(USExp, '\\US_'+timestamp+".xlsx")'''

'format_Exp_pivot(Developing_Economies_RollupExp, \'\\Developing_Economies_\'+timestamp+".xlsx")\nformat_Exp_pivot(IndiaExp, \'\\India_\'+timestamp+".xlsx")\nformat_Exp_pivot(OperationsExp, \'\\Operations_\'+timestamp+".xlsx")\nformat_Exp_pivot(ChinaExp, \'\\China_\'+timestamp+".xlsx")\nformat_Exp_pivot(Breakthrough_Technology_Exp, \'\\Breakthrough_Technology-\'+timestamp+".xlsx")\nformat_Exp_pivot(CarbonFree_MobilityExp, \'\\CarbonFree_Mobility_\'+timestamp+".xlsx")\nformat_Exp_pivot(ClimateAligned_IndustriesExp, \'\\Climate_Aligned_Industries_\'+timestamp+".xlsx")\nformat_Exp_pivot(GlobalClimate_FinanceExp, \'\\Global_Climate_Finance_\'+timestamp+".xlsx")\nformat_Exp_pivot(Strategic_Engagement_and_Analysis_GroupExp, \'\\Strategic_Engagement_and_Analysis_Group_\'+timestamp+".xlsx")\nformat_Exp_pivot(CarbonFreeBuildings_RollupExp, \'\\CarbonFree_Buildings_\'+timestamp+".xlsx")\nformat_Exp_pivot(Watttime_Exp, \'\\WattTime_\'+timestamp+".xlsx")\nformat_Exp_pivot(Carbon_Free_ElectricityEx

# Create Reports for Heather and Jon Creyts

In [120]:
CFOpath = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\reports\CFO_Reports'

## Restricted Revenue Spend Type by Grant

In [121]:
def create_CFO_pivot(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Grant','Inst or Reclass'], values = 'Restricted Revenue', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [122]:
def add_subtotal_CFO(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, 'Subtotal'))) for k, d in PivotedGroup.groupby('Grant')]).append((PivotedGroup.sum()).rename(('Grand', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [123]:
def to_excel_CFO(pivot, str):
    pivot.to_excel(CFOpath+str, freeze_panes = (1,2), engine = 'openpyxl', sheet_name = 'Expense Type by Grant', float_format = "%.2f")

In [124]:
def edit_workbook_CFO(file):
    book = load_workbook(file)
    sheet = book.active #active means last opened sheet
    sheet['A1']= "Grant"
    sheet['B1']= "Expense Type"
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(3, 16):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 3):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=2).value == "Subtotal":
            for j in range(3,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
#    sheet.protection.sheet = True
    book.save(file)

In [125]:
def format_multi_pivot_CFO(Group, str):
    step1 = create_CFO_pivot(Group)
    step2 = add_subtotal_CFO(step1)
    to_excel_CFO(step2, str)
    edit_workbook_CFO(CFOpath+str)

In [126]:
dfRestrictedONLY = df1[df1['Rev or Exp']=="Expense"]

In [127]:
format_multi_pivot_CFO(dfRestrictedONLY, '\\CFO_Report_'+timestamp+".xlsx")

## Unrestricted Use by Project Allocation and Program

In [128]:
def create_CFO_pivot2(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Project Allocations','Program'], values = 'Unrestricted', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [129]:
def add_subtotal_CFO2(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, 'Subtotal'))) for k, d in PivotedGroup.groupby('Project Allocations')]).append((PivotedGroup.sum()).rename(('Grand', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [130]:
def to_excel_CFO2(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Unrestricted by Project All")
    writer.save()
    writer.close()    

In [131]:
def edit_workbook_CFO2(file):
    book = load_workbook(file)
    sheet = book["Unrestricted by Project All"] #active means last opened sheet
    sheet['A1']= "Initiative"
    sheet['B1']= "Project"
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
 
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(3, 16):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 3):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=2).value == "Subtotal":
            for j in range(3,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
    sheet.freeze_panes = "B2"
    book.save(file)

In [132]:
def format_multi_pivot_CFO2(Group, str):
    step1 = create_CFO_pivot2(Group)
    step2 = add_subtotal_CFO2(step1)
    to_excel_CFO2(step2, CFOpath+str)
    edit_workbook_CFO2(CFOpath+str)

In [133]:
format_multi_pivot_CFO2(df1, '\\CFO_Report_'+timestamp+".xlsx")

# Managing Director Dashboard

In [134]:
MDpath = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\reports\MD_Package'


Templatepath = 'C:\\Users\\Jeremy Wendt\\anaconda3\\envs\\Code\\Detail_Trans_Project_FY21\\Data\\raw\\MD_Cover_Page.xlsx'

In [135]:

#wb = openpyxl.load_workbook(filename=Templatepath)
#ws = wb['Cover']

In [136]:
ProgramsMD = Programs

In [137]:
ProgramsMD.remove('No Program')
ProgramsMD.remove('Communications')
ProgramsMD.remove('Development')
ProgramsMD.remove('Building Electrification')
ProgramsMD.remove('Islands')
ProgramsMD.remove('SE Asia')
ProgramsMD.remove('Africa')
ProgramsMD.append('Developing Economies')

In [138]:
todayMD = datetime.today()
datemMD = datetime(today.year, today.month-1, 1)
timestampMD = datem.strftime("%B %Y")

In [139]:
timestampMD

'February 2021'

## Create Cover Sheet

In [140]:
CoverNames = []

for i in ProgramsMD:
    CoverNames.append(i + " - " + timestampMD)

In [141]:
Logopath = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\notebooks\img\Picture1.png'

# 'C:\\Users\\Jeremy Wendt\\anaconda3\\envs\\Code\\Detail_Trans_Project_FY21\\notebooks\\img\\MD_Cover_Page.xlsx'

In [142]:
for i in CoverNames:    
    wb = load_workbook(Templatepath)
    ws = wb['Cover']    
    img = openpyxl.drawing.image.Image(Logopath)
    img.anchor = 'A1'
    ws.add_image(img)
    ws.cell(row = 14, column = 1).value = i
    wb.save(str(MDpath+'\\'+i+'.xlsx'))

## Create First MD Table

In [143]:
blueFill = PatternFill(start_color='0033CCCC',
                   end_color='0033CCCC',
                   fill_type='solid')



In [144]:
def create_MD_pivot1(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [145]:
def add_subtotal_MD1(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k,'', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand','', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [146]:
def to_excel_MD1(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Restricted Revenue Exp", startrow = 1)
    writer.save()
    writer.close()    

In [147]:
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment

def edit_workbook_MD1(file):
    book = load_workbook(file)
    sheet = book["Restricted Revenue Exp"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Project"
    sheet['C2']= "Grant"    
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:P1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['P1']
    top_left_cell.value = "Restricted Revenue Expenses"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="DDDDDD")
    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="FFFFFF")
    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(4, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 4):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=3).value == "Subtotal":
            for j in range(3,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(17, 18):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "D3"
    book.save(file)

In [148]:
def format_multi_pivot_MD1(Group, str):
    step1 = create_MD_pivot1(Group)
    step2 = add_subtotal_MD1(step1)
    to_excel_MD1(step2, MDpath+str)
    edit_workbook_MD1(MDpath+str)

In [149]:
CoverNames

['India - February 2021',
 'Operations - February 2021',
 'China - February 2021',
 'Breakthrough Technology - February 2021',
 'Carbon-Free Mobility - February 2021',
 'Climate Aligned Industries - February 2021',
 'Global Climate Finance - February 2021',
 'Carbon-Free Electricity - February 2021',
 'Strategic Engagement and Analysis Group - February 2021',
 'Carbon-Free Buildings - February 2021',
 'WattTime - February 2021',
 'Urban Transformation - February 2021',
 'Climate Intelligence - February 2021',
 'Industry - February 2021',
 'US - February 2021',
 'Energy Transition Academy - February 2021',
 'Developing Economies - February 2021']

In [150]:
format_multi_pivot_MD1(Developing_Economies_Rollup, '\\Developing Economies - '+timestampMD+".xlsx")
format_multi_pivot_MD1(Climate_Aligned_Industries, '\\Climate Aligned Industries - '+timestampMD+".xlsx")
format_multi_pivot_MD1(India, '\\India - '+timestampMD+".xlsx")
format_multi_pivot_MD1(Operations, '\\Operations - '+timestampMD+".xlsx")
format_multi_pivot_MD1(China, '\\China - '+timestampMD+".xlsx")
format_multi_pivot_MD1(Breakthrough_Technology, '\\Breakthrough Technology - '+timestampMD+".xlsx")
format_multi_pivot_MD1(CarbonFree_Mobility, '\\Carbon-Free Mobility - '+timestampMD+".xlsx")
format_multi_pivot_MD1(CarbonFree_Electricty, '\\Carbon-Free Electricity - '+timestampMD+".xlsx")
format_multi_pivot_MD1(Strategic_Engagement_and_Analysis_Group, '\\Strategic Engagement and Analysis Group - '+timestampMD+".xlsx")
format_multi_pivot_MD1(CarbonFreeBuildings_Rollup, '\\Carbon-Free Buildings - '+timestampMD+".xlsx")
format_multi_pivot_MD1(Urban_Transformation, '\\Urban Transformation - '+timestampMD+".xlsx")
format_multi_pivot_MD1(Climate_Intelligence, '\\Climate Intelligence - '+timestampMD+".xlsx")
format_multi_pivot_MD1(Industry, '\\Industry - '+timestampMD+".xlsx")
format_multi_pivot_MD1(US, '\\US - '+timestampMD+".xlsx")


# Create Multi Lat Tab

In [151]:
def create_MD_pivot2(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Client'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [152]:
def add_subtotal_MD2(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k,'', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand','', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [153]:
def to_excel_MD2(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="ER & ML Govt Rev", startrow = 1)
    writer.save()
    writer.close()    

In [154]:
def edit_workbook_MD2(file):
    book = load_workbook(file)
    sheet = book["ER & ML Govt Rev"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Project"
    sheet['C2']= "Client"    
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:P1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['P1']
    top_left_cell.value = "Earned, Government, and Multilateral Grant Revenue"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="DDDDDD")
    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="FFFFFF")
    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(4, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 4):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=3).value == "Subtotal":
            for j in range(3,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(17, 18):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "D3"
    book.save(file)

In [155]:
def format_multi_pivot_MD2(Group, str):
    step1 = create_MD_pivot2(Group)
    step2 = add_subtotal_MD2(step1)
    to_excel_MD2(step2, MDpath+str)
    edit_workbook_MD2(MDpath+str)

In [156]:
format_multi_pivot_MD2(Developing_Economies_RollupFFS, '\\Developing Economies - '+timestampMD+".xlsx")
format_multi_pivot_MD2(ClimateAligned_IndustriesFFS, '\\Climate Aligned Industries - '+timestampMD+".xlsx")
format_multi_pivot_MD2(IndiaFFS, '\\India - '+timestampMD+".xlsx")
format_multi_pivot_MD2(OperationsFFS, '\\Operations - '+timestampMD+".xlsx")
format_multi_pivot_MD2(ChinaFFS, '\\China - '+timestampMD+".xlsx")
format_multi_pivot_MD2(Breakthrough_Technology_FFS, '\\Breakthrough Technology - '+timestampMD+".xlsx")
format_multi_pivot_MD2(CarbonFree_MobilityFFS, '\\Carbon-Free Mobility - '+timestampMD+".xlsx")
format_multi_pivot_MD2(Carbon_Free_ElectricityFFS, '\\Carbon-Free Electricity - '+timestampMD+".xlsx")
format_multi_pivot_MD2(Strategic_Engagement_and_Analysis_GroupFFS, '\\Strategic Engagement and Analysis Group - '+timestampMD+".xlsx")
format_multi_pivot_MD2(CarbonFreeBuildings_RollupFFS, '\\Carbon-Free Buildings - '+timestampMD+".xlsx")
format_multi_pivot_MD2(UrbanTrans_FFS, '\\Urban Transformation - '+timestampMD+".xlsx")
format_multi_pivot_MD2(ClimateIntelFFS, '\\Climate Intelligence - '+timestampMD+".xlsx")
format_multi_pivot_MD2(IndustryFFS, '\\Industry - '+timestampMD+".xlsx")
format_multi_pivot_MD2(USFFS, '\\US - '+timestampMD+".xlsx")

# Create All Expenses Report

In [157]:
def create_MD_pivot3(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [158]:
def add_subtotal_MD3(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k,'', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand','', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [159]:
def to_excel_MD3(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Total Expenses", startrow = 1)
    writer.save()
    writer.close()    

In [160]:
def edit_workbook_MD3(file):
    book = load_workbook(file)
    sheet = book["Total Expenses"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Project"
    sheet['C2']= "Grant"    
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:P1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['P1']
    top_left_cell.value = "Total Program Expenses by Initiative, Project, and Grant"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="DDDDDD")
    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="FFFFFF")
    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(4, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 4):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=3).value == "Subtotal":
            for j in range(3,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(17, 18):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "D3"
    book.save(file)

In [161]:
def format_multi_pivot_MD3(Group, str):
    step1 = create_MD_pivot3(Group)
    step2 = add_subtotal_MD3(step1)
    to_excel_MD3(step2, MDpath+str)
    edit_workbook_MD3(MDpath+str)

In [162]:
format_multi_pivot_MD3(Developing_Economies_RollupExp, '\\Developing Economies - '+timestampMD+".xlsx")
format_multi_pivot_MD3(ClimateAligned_IndustriesExp, '\\Climate Aligned Industries - '+timestampMD+".xlsx")
format_multi_pivot_MD3(IndiaExp, '\\India - '+timestampMD+".xlsx")
format_multi_pivot_MD3(OperationsExp, '\\Operations - '+timestampMD+".xlsx")
format_multi_pivot_MD3(ChinaExp, '\\China - '+timestampMD+".xlsx")
format_multi_pivot_MD3(Breakthrough_Technology_Exp, '\\Breakthrough Technology - '+timestampMD+".xlsx")
format_multi_pivot_MD3(CarbonFree_MobilityExp, '\\Carbon-Free Mobility - '+timestampMD+".xlsx")
format_multi_pivot_MD3(Carbon_Free_ElectricityExp, '\\Carbon-Free Electricity - '+timestampMD+".xlsx")
format_multi_pivot_MD3(Strategic_Engagement_and_Analysis_GroupExp, '\\Strategic Engagement and Analysis Group - '+timestampMD+".xlsx")
format_multi_pivot_MD3(CarbonFreeBuildings_RollupExp, '\\Carbon-Free Buildings - '+timestampMD+".xlsx")
format_multi_pivot_MD3(UrbanTrans_Exp, '\\Urban Transformation - '+timestampMD+".xlsx")
format_multi_pivot_MD3(ClimateIntelExp, '\\Climate Intelligence - '+timestampMD+".xlsx")
format_multi_pivot_MD3(IndustryExp, '\\Industry - '+timestampMD+".xlsx")
format_multi_pivot_MD3(SEAsiaExp, '\\SE Asia - '+timestampMD+".xlsx")
format_multi_pivot_MD3(USExp, '\\US - '+timestampMD+".xlsx")
format_multi_pivot_MD3(BuildingElectrificationExp, '\\Building Electrification - '+timestampMD+".xlsx")

# Unrestricted Report

In [163]:
def create_Totalprogram(program):
    firstdf = df1[df1['Program']==program]
    if firstdf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return firstdf

In [164]:
DevelopmentTotal = create_Totalprogram('Development')
IslandsTotal = create_Totalprogram('Islands')
IndiaTotal = create_Totalprogram('India')
OperationsTotal = create_Totalprogram('Operations')
ChinaTotal = create_Totalprogram('China')
Breakthrough_TechnologyTotal = create_Totalprogram('Breakthrough Technology')
CarbonFree_MobilityTotal = create_Totalprogram('Carbon-Free Mobility')
Climate_Aligned_IndustriesTotal = create_Totalprogram('Climate Aligned Industries')
Global_Climate_FinanceTotal = create_Totalprogram('Global Climate Finance')
Strategic_Engagement_and_Analysis_GroupTotal = create_Totalprogram('Strategic Engagement and Analysis Group')
CarbonFree_BuildingsTotal = create_Totalprogram('Carbon-Free Buildings')
WattTimeTotal = create_Totalprogram('WattTime')
CarbonFree_ElectrictyTotal = create_Totalprogram('Carbon-Free Electricity')
Urban_TransformationTotal = create_Totalprogram('Urban Transformation')
Climate_IntelligenceTotal = create_Totalprogram('Climate Intelligence')
IndustryTotal = create_Totalprogram('Industry')
SE_AsiaTotal = create_Totalprogram('SE Asia')
AfricaTotal = create_Totalprogram('Africa')
USTotal = create_Totalprogram('US')
Building_ElectrificationTotal = create_Totalprogram('Building Electrification')

In [165]:
Developing_Economies_RollupTotal = pd.concat([IslandsTotal, AfricaTotal, SE_AsiaTotal], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupTotal = pd.concat([CarbonFree_BuildingsTotal, Building_ElectrificationTotal], ignore_index = True, sort=False)

In [166]:
def create_MD_pivot4(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant'], values = 'Unrestricted', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [167]:
def add_subtotal_MD4(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k,'', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand','', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [168]:
def to_excel_MD4(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Unrestricted Use", startrow = 1)
    writer.save()
    writer.close()    

In [169]:
def edit_workbook_MD4(file):
    book = load_workbook(file)
    sheet = book["Unrestricted Use"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Project"
    sheet['C2']= "Grant"    
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:P1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['P1']
    top_left_cell.value = "Unrestricted Use by Initiative, Project, and Grant"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="DDDDDD")
    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="FFFFFF")
    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(4, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 4):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=3).value == "Subtotal":
            for j in range(3,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin) 
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(17, 18):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "D3"
    book.save(file)

In [170]:
def format_multi_pivot_MD4(Group, str):
    step1 = create_MD_pivot4(Group)
    step2 = add_subtotal_MD4(step1)
    to_excel_MD4(step2, MDpath+str)
    edit_workbook_MD4(MDpath+str)

In [171]:
format_multi_pivot_MD4(Developing_Economies_RollupTotal, '\\Developing Economies - '+timestampMD+".xlsx")
format_multi_pivot_MD4(Climate_Aligned_IndustriesTotal, '\\Climate Aligned Industries - '+timestampMD+".xlsx")
format_multi_pivot_MD4(IndiaTotal, '\\India - '+timestampMD+".xlsx")
format_multi_pivot_MD4(OperationsTotal, '\\Operations - '+timestampMD+".xlsx")
format_multi_pivot_MD4(ChinaTotal, '\\China - '+timestampMD+".xlsx")
format_multi_pivot_MD4(Breakthrough_TechnologyTotal, '\\Breakthrough Technology - '+timestampMD+".xlsx")
format_multi_pivot_MD4(CarbonFree_MobilityTotal, '\\Carbon-Free Mobility - '+timestampMD+".xlsx")
format_multi_pivot_MD4(CarbonFree_ElectrictyTotal, '\\Carbon-Free Electricity - '+timestampMD+".xlsx")
format_multi_pivot_MD4(Strategic_Engagement_and_Analysis_GroupTotal, '\\Strategic Engagement and Analysis Group - '+timestampMD+".xlsx")
format_multi_pivot_MD4(CarbonFreeBuildings_RollupTotal, '\\Carbon-Free Buildings - '+timestampMD+".xlsx")
format_multi_pivot_MD4(Urban_TransformationTotal, '\\Urban Transformation - '+timestampMD+".xlsx")
format_multi_pivot_MD4(Climate_IntelligenceTotal, '\\Climate Intelligence - '+timestampMD+".xlsx")
format_multi_pivot_MD4(IndustryTotal, '\\Industry - '+timestampMD+".xlsx")
format_multi_pivot_MD4(SE_AsiaTotal, '\\SE Asia - '+timestampMD+".xlsx")
format_multi_pivot_MD4(USTotal, '\\US - '+timestampMD+".xlsx")
format_multi_pivot_MD4(Building_ElectrificationTotal, '\\Building Electrification - '+timestampMD+".xlsx")

# Mavenlink Report

In [172]:
def create_Mavenlinkprogram(program):
    firstdf = dfMavenlink[dfMavenlink['Program']==program]
    if firstdf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return firstdf

In [173]:
DevelopmentML = create_Mavenlinkprogram("Development")
IslandsML = create_Mavenlinkprogram("Islands")
IndiaML = create_Mavenlinkprogram("India")
OperationsML = create_Mavenlinkprogram("Operations")
ChinaML = create_Mavenlinkprogram("China")
Breakthrough_Technology_ML = create_Mavenlinkprogram("Breakthrough Technology")
CarbonFree_MobilityML = create_Mavenlinkprogram("Carbon-Free Mobility")
ClimateAligned_IndustriesML = create_Mavenlinkprogram("Climate Aligned Industries")
CommunicationsML = create_Mavenlinkprogram("Communications")
GlobalClimate_FinanceML = create_Mavenlinkprogram("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupML = create_Mavenlinkprogram("Strategic Engagement and Analysis Group")
Carbon_Free_BuildingsML = create_Mavenlinkprogram("Carbon-Free Buildings")
Carbon_Free_ElectricityML = create_Mavenlinkprogram("Carbon-Free Electricity")
UrbanTrans_ML = create_Mavenlinkprogram("Urban Transformation")
ClimateIntelML = create_Mavenlinkprogram("Climate Intelligence")
IndustryML = create_Mavenlinkprogram("Industry")
SEAsiaML = create_Mavenlinkprogram("SE Asia")
AfricaML = create_Mavenlinkprogram("Africa")
USML = create_Mavenlinkprogram("US")
BuildingElectrificationML = create_Mavenlinkprogram("Building Electrification")

In [175]:
Developing_Economies_RollupML = pd.concat([IslandsML, AfricaML, SEAsiaML], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupML = pd.concat([Carbon_Free_BuildingsML, BuildingElectrificationML], ignore_index = True, sort=False)

In [176]:
def create_MD_pivot5(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Employee','Initiative','Case/Task/Event','Grant','Billable'], values = 'Hours', columns = 'Month', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [177]:
def add_subtotal_MD5(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k,'','','','Subtotal'))) for k, d in PivotedGroup.groupby('Employee')]).append((PivotedGroup.sum()).rename(('Grand','','','','Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [178]:
def to_excel_MD5(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Mavenlink Hours", startrow = 1)
    writer.save()
    writer.close()   

In [179]:
def edit_workbook_MD5(file):
    book = load_workbook(file)
    sheet = book["Mavenlink Hours"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Employee"
    sheet['B2']= "Initiative"
    sheet['C2']= "Case/Task/Event"    
    sheet['D2']= "Grant"     
    sheet['E2']= "Billable"    
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True)
    sheet['E2'].font = Font(bold=True)
    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:R1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['R1']
    top_left_cell.value = "Mavenlink Hours by Employee, Initiative, Case/Task/Event (Excludes Time Adjustments)"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="DDDDDD")
    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="FFFFFF")
    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 25
    sheet.column_dimensions['E'].width = 25
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
    sheet.column_dimensions['Q'].width = 15
    sheet.column_dimensions['R'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(6, 19):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '#,##0;[Red](#,##0)'
    for col in range(1, 6):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=5).value == "Subtotal":
            for j in range(5,19):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,19):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Employee":
            for j in range(1,19):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(19, 20):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 60
    sheet.freeze_panes = "F3"
    book.save(file)

In [180]:
def format_multi_pivot_MD5(Group, str):
    step1 = create_MD_pivot5(Group)
    step2 = add_subtotal_MD5(step1)
    to_excel_MD5(step2, MDpath+str)
    edit_workbook_MD5(MDpath+str)

In [181]:
format_multi_pivot_MD5(Developing_Economies_RollupML, '\\Developing Economies - '+timestampMD+".xlsx")
format_multi_pivot_MD5(ClimateAligned_IndustriesML, '\\Climate Aligned Industries - '+timestampMD+".xlsx")
format_multi_pivot_MD5(IndiaML, '\\India - '+timestampMD+".xlsx")
format_multi_pivot_MD5(OperationsML, '\\Operations - '+timestampMD+".xlsx")
format_multi_pivot_MD5(ChinaML, '\\China - '+timestampMD+".xlsx")
format_multi_pivot_MD5(Breakthrough_Technology_ML, '\\Breakthrough Technology - '+timestampMD+".xlsx")
format_multi_pivot_MD5(CarbonFree_MobilityML, '\\Carbon-Free Mobility - '+timestampMD+".xlsx")
format_multi_pivot_MD5(Carbon_Free_ElectricityML, '\\Carbon-Free Electricity - '+timestampMD+".xlsx")
format_multi_pivot_MD5(Strategic_Engagement_and_Analysis_GroupML, '\\Strategic Engagement and Analysis Group - '+timestampMD+".xlsx")
format_multi_pivot_MD5(CarbonFreeBuildings_RollupML, '\\Carbon-Free Buildings - '+timestampMD+".xlsx")
format_multi_pivot_MD5(UrbanTrans_ML, '\\Urban Transformation - '+timestampMD+".xlsx")
format_multi_pivot_MD5(ClimateIntelML, '\\Climate Intelligence - '+timestampMD+".xlsx")
format_multi_pivot_MD5(IndustryML, '\\Industry - '+timestampMD+".xlsx")
format_multi_pivot_MD5(SEAsiaML, '\\SE Asia - '+timestampMD+".xlsx")
format_multi_pivot_MD5(USML, '\\US - '+timestampMD+".xlsx")
format_multi_pivot_MD5(BuildingElectrificationML, '\\Building Electrification - '+timestampMD+".xlsx")


In [182]:
dfMavenlink.head(35)

,Internal ID,External ID,Employee,Duration,Program,Initiative,Client,Grant,Restriction,Case/Task/Event,Approval Status,Billable,Date,Item,Type,Actual Time Entry Cost,Month,Minutes,Hours
0,9607850,1535266785,Radhika Lalit,8:00,Global Climate Finance,GCF - Center for a Climate-Aligned Financial S...,*Restricted : Center for Climate Aligned Finance,Wells Fargo - GCF CCAF FY20,1-Unrestricted,Admin (Project Task),Approved,Yes,2020-07-01T00:00:00,Professional Personnel,Actual Time,631.861,Jul 2020,480,8.0
1,9609942,1536835065,Emily Goldfield,4:00,India,IN - India,Children's Investment Fund Foundation : CIFF T...,No Grant,1-Unrestricted,Project management (Project Task),Approved,Yes,2020-07-01T00:00:00,Professional Personnel,Actual Time,264.776,Jul 2020,240,4.0
2,9627632,1531726455,Holly Leonard,4:00,Urban Transformation,UT - Admin,*Unrestricted : Urban Transformation [Holiday]...,Bloomberg America's Pledge FY20,1-Unrestricted,PTO-AP2020 (Project Task),Approved,No,2020-07-01T00:00:00,Professional Personnel,Actual Time,230.71,Jul 2020,240,4.0
3,9593084,1532866255,James Newcomb,2:00,Strategic Engagement and Analysis Group,SEA - Admin,*Unrestricted : Strategic Engagement and Analy...,Wells Fargo - EES Phase 2 FY20,1-Unrestricted,Meetings (Project Task),Approved,No,2020-07-01T00:00:00,Professional Personnel,Actual Time,453.404,Jul 2020,120,2.0
4,9606292,1535531785,Shelley Backstrom,2:00,Islands,IS - Islands,*Restricted : Bermuda,Anonymous - Bermuda FY20,1-Unrestricted,Electricity (Project Task),Approved,Yes,2020-07-01T00:00:00,Professional Personnel,Actual Time,104.538,Jul 2020,120,2.0
5,9627081,1532348615,Brian O'Hanlon,8:00,Global Climate Finance,GCF - Center for a Climate-Aligned Financial S...,*Restricted : Center for Climate Aligned Finance,Wells Fargo - GCF CCAF FY20,1-Unrestricted,General Strategy and Planning (Project Task),Approved,Yes,2020-07-01T00:00:00,Professional Personnel,Actual Time,1029.293,Jul 2020,480,8.0
6,9629076,1531693045,Clay Stranger,4:00,India,IN - Admin,*Unrestricted : India [Holiday] [PTO],No Grant,1-Unrestricted,Holiday (Project Task),Approved,No,2020-07-01T00:00:00,Professional Personnel,Actual Time,531.064,Jul 2020,240,4.0
7,9627615,1532113575,Garrett Fitzgerald,2:00,India,IN - India,Shell International B.V. : India - Electricity...,No Grant,1-Unrestricted,Phase 2 - LH Discom Rajasthan (Project Task),Approved,Yes,2020-07-01T00:00:00,Professional Personnel,Actual Time,166.012,Jul 2020,120,2.0
8,9629356,1529936695,Nicole Martin,8:00,Operations,OPS - People Team,*Unrestricted : Human Resources [Admin],No Grant,1-Unrestricted,General (Project Task),Approved,No,2020-07-01T00:00:00,Professional Personnel,Actual Time,657.963,Jul 2020,480,8.0
9,9605978,1535558315,Jon Creyts,4:00,Operations,OPS - CEO Special Projects,*Unrestricted : Project Chrysalis,No Grant,1-Unrestricted,Operationalizing the Transition (Project Task),Approved,Yes,2020-07-01T00:00:00,Professional Personnel,Actual Time,944.429,Jul 2020,240,4.0


## Create Salesforce Pivots

In [183]:
def create_Salesforceprogram(program):
    firstdf = dfSalesforce[dfSalesforce['Program: Program Name']==program]
    if firstdf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return firstdf

In [184]:
DevelopmentSF = create_Salesforceprogram("General Support")
IslandsSF = create_Salesforceprogram("Islands")
IndiaSF = create_Salesforceprogram("India")
OperationsSF = create_Salesforceprogram("Operations")
ChinaSF = create_Salesforceprogram("China")
Breakthrough_Technology_SF = create_Salesforceprogram("Breakthrough Technology")
CarbonFree_MobilitySF = create_Salesforceprogram("Carbon-Free Mobility")
ClimateAligned_IndustriesSF = create_Salesforceprogram("Climate Aligned Industries")
#CommunicationsSF = create_Salesforceprogram("Communications")
GlobalClimate_FinanceSF = create_Salesforceprogram("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupSF = create_Salesforceprogram("Strategic Engagement and Analysis Group")
Carbon_Free_BuildingsSF = create_Salesforceprogram("Carbon-Free Buildings")
Carbon_Free_ElectricitySF = create_Salesforceprogram("Carbon-Free Electricity")
UrbanTrans_SF = create_Salesforceprogram("Urban Transformation")
ClimateIntelSF = create_Salesforceprogram("Climate Intelligence")
IndustrySF = create_Salesforceprogram("Industry")
SEAsiaSF = create_Salesforceprogram("SE Asia Energy")
AfricaSF = create_Salesforceprogram("Africa")
USSF = create_Salesforceprogram("US")
BuildingElectrificationSF = create_Salesforceprogram("Building Electrification")

Building Electrification DF is EMPTY


In [185]:
Developing_Economies_RollupSF = pd.concat([IslandsSF, AfricaSF, SEAsiaSF], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupSF = pd.concat([Carbon_Free_BuildingsSF, BuildingElectrificationSF], ignore_index = True, sort=False)

In [186]:
def create_MD_pivot6(Group):
    table = pd.pivot_table(Group, index = ['Initiative','Forecast Category','Opportunity Name','Revenue Name','Stage','Close Date','Revenue Status', 'Program Fund Type', 'Probability (%)', 'Revenue Amount'], values = 'Expected Revenue', aggfunc=np.sum)
    return table

In [187]:
today = datetime.today()
timestamptoday = today.strftime("%B %d, %Y")

In [188]:
def to_excel_MD6(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Salesforce Data", startrow = 1)
    writer.save()
    writer.close()   

In [189]:
def edit_workbook_MD6(file):
    book = load_workbook(file)
    sheet = book["Salesforce Data"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['C2']= "Opportunity Name"
    sheet['D2']= "Revenue Name"    
    sheet['B2']= "Forecast Category" 
    sheet['E2']= "Stage"    
    sheet['F2']= "Close Date"   
    sheet['G2']= "Revenue Status"   
    sheet['H2']= "Program Fund Type"   
    sheet['I2']= "Probability (%)"     
    sheet['J2']= "Revenue Amount"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:K1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['K1']
    top_left_cell.value = "SalesForce Data as of " +timestamptoday
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="DDDDDD")
    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(b=True, color="FFFFFF")
    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 25
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 25
    sheet.column_dimensions['G'].width = 25
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(9, 12):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '#,##0;[Red](#,##0)'
    for col in range(1, 5):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,12):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(11, 12):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(top=thin, left=thin, right=thin, bottom=thin)
    for col in range(4, 12):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).font = Font(bold=False)
    sheet.sheet_view.zoomScale = 85  
    sheet.freeze_panes = sheet['D3']
    book.save(file)

In [190]:
def format_multi_pivot_MD6(Group, str):
    step1 = create_MD_pivot6(Group)
    to_excel_MD6(step1, MDpath+str)
    edit_workbook_MD6(MDpath+str)

In [191]:
format_multi_pivot_MD6(Developing_Economies_RollupSF, '\\Developing Economies - '+timestampMD+".xlsx")
format_multi_pivot_MD6(ClimateAligned_IndustriesSF, '\\Climate Aligned Industries - '+timestampMD+".xlsx")
format_multi_pivot_MD6(IndiaSF, '\\India - '+timestampMD+".xlsx")
format_multi_pivot_MD6(OperationsSF, '\\Operations - '+timestampMD+".xlsx")
format_multi_pivot_MD6(ChinaSF, '\\China - '+timestampMD+".xlsx")
format_multi_pivot_MD6(Breakthrough_Technology_SF, '\\Breakthrough Technology - '+timestampMD+".xlsx")
format_multi_pivot_MD6(CarbonFree_MobilitySF, '\\Carbon-Free Mobility - '+timestampMD+".xlsx")
format_multi_pivot_MD6(Carbon_Free_ElectricitySF, '\\Carbon-Free Electricity - '+timestampMD+".xlsx")
format_multi_pivot_MD6(Strategic_Engagement_and_Analysis_GroupSF, '\\Strategic Engagement and Analysis Group - '+timestampMD+".xlsx")
format_multi_pivot_MD6(CarbonFreeBuildings_RollupSF, '\\Carbon-Free Buildings - '+timestampMD+".xlsx")
format_multi_pivot_MD6(UrbanTrans_SF, '\\Urban Transformation - '+timestampMD+".xlsx")
format_multi_pivot_MD6(ClimateIntelSF, '\\Climate Intelligence - '+timestampMD+".xlsx")
format_multi_pivot_MD6(IndustrySF, '\\Industry - '+timestampMD+".xlsx")
format_multi_pivot_MD6(USSF, '\\US - '+timestampMD+".xlsx")
#format_multi_pivot_MD6(BuildingElectrificationSF, '\\Building Electrification - '+timestampMD+".xlsx")